In [2]:
import math
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import defs
import requests
# from backtesting.lib import crossover


### Trading Data

In [3]:
# Importing through a CSV that has more data which was downloaded externally.
df = pd.read_csv('../Data/AUDCAD_H1(22007-09-26 - 2024-02-02).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
df.index = pd.to_datetime(df.index)


In [ ]:
df.drop(columns='Volume', axis=1, inplace=True)

### Trades DF initialisation

In [ ]:
trades_columns = ['Open Date', 'Close Date','Direction', 'Stop Loss (Pips)', 'Closing Result(Pips)', 'R', 'Equity', 'PnL_pct']
trades_df = pd.DataFrame(columns=trades_columns)

In [ ]:
df.columns

## Initialise Indicators here

In [ ]:
df.head()

In [ ]:
df['Signal'] = ''
df['Position'] = False

In [ ]:
# EMA and ATR with defaults length @ 14

df['ATR'] = ta.atr(pd.Series(df.High),
                   pd.Series(df.Low),
                   pd.Series(df.Close),
                   length=14)

df['EMA50'] = ta.ema(pd.Series(df.Close), length=50)

## Trading Account info for testing

In [ ]:
initial_balance = 100000
equity = initial_balance
open_position = False
trade_history = []

## Row itteration

In [ ]:
df.tail()

In [ ]:
test_df = df.iloc[100:150].copy()
test_df['Stop'] = 0
test_df['Stop pips'] = 0
test_df['Take Profit'] = 0

In [ ]:
test_df.head()

SL and TP for BUY orders

In [ ]:
test_df.head()

In [ ]:
round(pips, 4)

In [ ]:
sl = test_df.Low.iloc[1] - test_df.ATR.iloc[1]
pips = round(test_df.Open.iloc[1] - test_df.Stop.iloc[1], 4)

test_df.iloc[1, test_df.columns.get_loc('Stop')] = sl
test_df.iloc[1, test_df.columns.get_loc('Stop pips')] = pips
rr = 1.2
test_df.iloc[1, test_df.columns.get_loc('Take Profit')] = test_df.Close.iloc[1] + (pips * rr)

## Price Variables

In [ ]:
# Price  variables

price = df.Close.iloc[-1]
ema = df.EMA50.iloc[-1]

current_open = df.Open.iloc[-1]
current_high = df.High.iloc[-1]
current_price = df.Close.iloc[-1]
previous_price = df.Close.iloc[-2]
previous_high = df.High.iloc[-2]
current_low = df.Low.iloc[-1]
current_atr = df.ATR.iloc[-1]

In [ ]:
previous_price


## Trade Signal

In [ ]:
def crossover(series1, series2):
    diff = series1 - series2
    prev_diff = diff.shift(1)
    bullish_crossover = (diff > 0) & (prev_diff <= 0)
    return bullish_crossover

df['Bullish_Crossover'] = crossover(df['Close'], df['EMA50'])


In [ ]:
# Note buy signals have been shifted 1 row to buy on the next candle open 
def buy(df, r):
    entry = df.Open.iloc[-1]
    sl = df.Low.iloc[-2] - df.ATR.iloc[-2]
    stop_pips = round(df.Open.iloc[-1] - sl)
    rr = stop_pips * r
    tp = entry + rr
    return entry, sl, tp


In [ ]:
df.tail()

In [9]:
import pandas as pd

# Assuming 'df' is your DataFrame and it includes columns for 'Open', 'High', 'Low', 'Close'

# Setup for trades log DataFrame
trades_columns = ['Open Date', 'Close Date', 'Direction', 'Stop Loss (Pips)', 'Closing Result (Pips)']
trades_df = pd.DataFrame(columns=trades_columns)

# Variables to track consolidation and pullback
consolidation_start = None
consolidation_high = None
consolidation_low = None
pullback_count = 0

trade_details_list = []

# Assigns current and previous row variables
for i in range(1, len(df)):
    current = df.iloc[i]
    previous = df.iloc[i - 1]

    # Check for consolidation continuation
    if consolidation_start is not None:
        if current['High'] <= consolidation_high and current['Close'] < current['Open']:
            pullback_count += 1  # Increment pullback count
        else:
            # Check for breakout condition
            if current['Close'] > previous['High'] and current['High'] <= consolidation_high:
                # BUY condition met, collect trade details
                entry_date = current.name
                entry_price = current['Open']
                sl = consolidation_low - (consolidation_high - consolidation_low)
                tp = entry_price + 2 * (entry_price - sl)
                close_date = entry_date  # Placeholder for actual trade closing logic
                closing_result_pips = tp - entry_price  # Placeholder for actual outcome calculation
                
                trade_details_list.append({
                    'Open Date': entry_date,
                    'Close Date': close_date,
                    'Direction': 'BUY',
                    'Stop Loss (Pips)': entry_price - sl,
                    'Closing Result (Pips)': closing_result_pips
                })
                
                # Reset variables for next iteration
                consolidation_start = None
                pullback_count = 0
            else:
                # If not a pullback or breakout, reset consolidation
                consolidation_start = None
                pullback_count = 0
    else:
        if current['High'] <= previous['High'] and current['Close'] < current['Open']:
            consolidation_start = i
            consolidation_high = current['High']
            consolidation_low = current['Low']
            pullback_count = 1  # Start counting pullbacks

# After the loop, convert the list of dictionaries to a DataFrame and concatenate with the original trades_df
new_trades_df = pd.DataFrame(trade_details_list, columns=trades_columns)
trades_df = pd.concat([trades_df, new_trades_df], ignore_index=True)

C:\Users\saleg\AppData\Local\Temp\ipykernel_21292\2299793979.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trades_df = pd.concat([trades_df, new_trades_df], ignore_index=True)


In [12]:
trades_df

,Open Date,Close Date,Direction,Stop Loss (Pips),Closing Result (Pips)
0,2007-10-01 09:00:00,2007-10-01 09:00:00,BUY,0.00100,0.00200
1,2007-10-24 06:00:00,2007-10-24 06:00:00,BUY,0.00068,0.00136
2,2007-11-02 19:00:00,2007-11-02 19:00:00,BUY,-0.00800,-0.01600
3,2007-11-26 08:00:00,2007-11-26 08:00:00,BUY,0.00096,0.00192
4,2007-12-28 04:00:00,2007-12-28 04:00:00,BUY,0.00178,0.00356
...,...,...,...,...,...
1240,2024-01-17 16:00:00,2024-01-17 16:00:00,BUY,0.00076,0.00152
1241,2024-01-19 17:00:00,2024-01-19 17:00:00,BUY,0.00125,0.00250
1242,2024-01-23 12:00:00,2024-01-23 12:00:00,BUY,-0.00113,-0.00226
1243,2024-01-30 05:00:00,2024-01-30 05:00:00,BUY,-0.00017,-0.00034
